In [459]:
import numpy as np
from numpy.random import randint
Apple_size = {
    1: "large size",
    -1: "small size"
}

Apple_smell = {
    1: "strong aroma",
    -1: "musty or rotten"
}

Apple_taste = {
    1: "more sweet than sour",
    -1: "more sour than sweet"
}

Apple_color = {
    1: "vibrant and uniformly colored",
    -1: "a large number of spots or browning on the surface"
}

Apple_juiciness = {
    1: "abundant and refreshing moisture",
    -1: "dry and lacking Moisture"
}

Apple_recmd = {
    1: "has significant market potential and deserves wider recognition",
    -1: "might not bring the expected returns and could even lead to losses",

}

map_0_1 = lambda x: 1 if x == 1 else -1

# def get_apple():
#   size = map_0_1(np.random.uniform() < 0.5)
#   smell = randint(3)- 1
#   taste_prob = 0.45 + 0.1*np.random.uniform() - 0.2*size 
#   taste = map_0_1(np.random.uniform() < taste_prob )
#   fea = np.array([size, smell, taste]) 
#   return fea

# def get_att():
#   while True:
#     size= np.random.uniform() < 1
#     smell = np.random.uniform() < 1
#     taste = np.random.uniform() < 1
#     fea = np.array([size, smell, taste]) 
#     if fea @ fea > 0:
#       break
#   return fea

def get_fea():
  
  size = map_0_1(np.random.uniform() < 0.5) * (np.random.uniform() < 0.9) * 1

  color = map_0_1(np.random.uniform() < 0.5) * (np.random.uniform() < 0.9) * 1  
  
  smell = map_0_1(np.random.uniform() < 0.5) * (np.random.uniform() < 0.9) * 1

  taste = map_0_1(np.random.uniform() < 0.5 ) * (np.random.uniform() < 0.9) * 1

  juiciness_prob = 0.4 + 0.6*(taste+1)/2 
  juiciness = map_0_1(np.random.uniform() < juiciness_prob ) * (np.random.uniform() < 0.9) * 1

  score = taste + size + smell + map_0_1(np.random.uniform() < 0.5) * (np.random.uniform() < 0.2) * 1

  # if score > 0:
  #   recmd_prob = 0.2*(score) + 0.3*(color)
  #   recmd = (np.random.uniform() < recmd_prob ) * (np.random.uniform() < 0.9) * 1
  # elif score < 0:
  #   recmd_prob = 0.2*(-score) + 0.3*(-color)
  #   recmd = - (np.random.uniform() < recmd_prob ) * (np.random.uniform() < 0.9) * 1
  # else:
  #   recmd = 0

  recmd_prob =  0.4 * score + 0.4 * (color+1)/2 # * smell size (color) 
  recmd = map_0_1(np.random.uniform() < recmd_prob ) * (np.random.uniform() < 0.9) * 1

  fea = np.array([score, color, size, smell, taste, juiciness, recmd]) 
  return fea


In [415]:
names = ['score', 'color', 'size', 'smell', 'taste', 'juiciness', 'recmd']

In [465]:
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.GraphUtils import GraphUtils
from copy import  deepcopy

while True:
    values = np.stack([get_fea() for _ in range(100)])
    # Causal Discovery (FCI)
    g, edges = fci(values, alpha = 0.05, verbose=False)

    cond = True 
    cond &= g.graph[0,2] == 1
    cond &= g.graph[0,3] == 1
    cond &= g.graph[0,4] == 1
    cond &= g.graph[3,0] != 1
    cond &= g.graph[4,0] != 1
    cond &= g.graph[6,0] == 1
    cond &= g.graph[6,1] == 1

    if cond:
        break
# visualization
pdy = GraphUtils.to_pydot(g, labels=names)
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 612.38it/s]

X1 --> X7
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=color];
2 [label=X3];
2 [label=size];
3 [label=X4];
3 [label=smell];
4 [label=X5];
4 [label=taste];
5 [label=X6];
5 [label=juiciness];
6 [label=X7];
6 [label=recmd];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 6  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [467]:
# ['score'  'color' 'size'  'smell' 'taste' 'juiciness' 'recmd']
#  0        1       2       3       4       5           6
idces = [0, 2, 3, 5, 6]

# Causal Discovery (FCI)
g, edges = fci(values[:,idces], alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=np.array(names)[idces])
print(pdy.to_string())

Depth=0, working on node 4: 100%|██████████| 5/5 [00:00<00:00, 248.91it/s]

X1 --> X5
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=size];
2 [label=X3];
2 [label=smell];
3 [label=X4];
3 [label=juiciness];
4 [label=X5];
4 [label=recmd];
1 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 4  [arrowhead=normal, arrowtail=none, dir=both];
}



In [41]:
# ['score'  'color' 'size'  'smell' 'taste' 'juiciness' 'recmd']
#  0        1       2       3       4       5           6
idces = [0, 5, 6]

# Causal Discovery (FCI)
g, edges = fci(values[:,idces], alpha = 0.05, verbose=False)

# visualization
pdy = GraphUtils.to_pydot(g, labels=np.array(names)[idces])
print(pdy.to_string())

Depth=0, working on node 2: 100%|██████████| 3/3 [00:00<00:00, 1023.08it/s]

digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=juiciness];
2 [label=X3];
2 [label=recmd];
0 -> 2  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [31]:
import pandas as pd

In [468]:
pd.DataFrame(values, columns=names)

,score,color,size,smell,taste,juiciness,recmd
0,-2,1,0,-1,-1,-1,-1
1,-1,1,1,-1,-1,1,-1
2,2,1,1,1,1,1,1
3,0,1,-1,1,-1,-1,1
4,-1,1,-1,-1,1,1,0
...,...,...,...,...,...,...,...
95,1,-1,1,-1,1,1,1
96,2,-1,1,-1,1,1,1
97,2,-1,1,0,1,1,1
98,3,-1,1,1,1,1,1


In [469]:
def get_prompt(score, color, size, smell, taste, juiciness, recmd):
  # conifig
  # You are an excellent story writter, full of imagination. You will finish your next masterpiece based on the given materials.
  introduction = 'You are a picky gastronome on apples with your own standard.'
  Reasons_adjustment = 'write one concrete and interesting reseaon.' # 'write one concrete reseaon.'
  Description_adjustment = 'with creative description.' #'with creative, vivid, diverse and detailed words. Avoid cliche.' # 'with creative and detailed words.'


  apple_feature_list = []

  for fea_v, fea_dict in zip(
    [color, size, smell, taste, juiciness, recmd],
    [Apple_color, Apple_size, Apple_smell, Apple_taste, Apple_juiciness, Apple_recmd]
    ):
    if fea_v in fea_dict.keys():
      apple_feature_list.append('- '+fea_dict[fea_v])
  apple_feature = '\n'.join(apple_feature_list)

  authors = ['Shakespeare', 'a musician', 'a poet', 'master Yoda']
  prompt = f'''

You are a picky gastronome on apples. You are ready to **evaluate apples and write reviews**. Your writing should be **clear, solid and convincing to suppliers and customers**.

##  Task

Please write a short reveiw about the evaluation results for a given apple.

Evaluation Results:
{apple_feature}


Additional Requirement:
- Combine those evaluation results into more detailed reasoning.
- Single paragraph; No quotation marks.
- Modern English
- Begin this part by a mark [Review Begin].
- no more than 60 words.

  '''
  return prompt

In [470]:
print(get_prompt(*values[0]))



You are a picky gastronome on apples. You are ready to **evaluate apples and write reviews**. Your writing should be **clear, solid and convincing to suppliers and customers**.

##  Task

Please write a short reveiw about the evaluation results for a given apple.

Evaluation Results:
- vibrant and uniformly colored
- musty or rotten
- more sour than sweet
- dry and lacking Moisture
- might not bring the expected returns and could even lead to losses


Additional Requirement:
- Combine those evaluation results into more detailed reasoning.
- Single paragraph; No quotation marks.
- Modern English
- Begin this part by a mark [Review Begin].
- no more than 60 words.

  


In [471]:
import os
import sys
sys.path.insert(0, '~/COAT') 

from utils import logger
from openai_utils import LLM_openai

import time

logger = Logger(
    root = '~/COAT/AppleGastronome', 
    exp_name = 'build_AG7_0102'
)

llm = LLM_openai(logger)

In [473]:
reviews = []
sample_size = 100


for idx in range(sample_size):
    prompt =get_prompt(*values[idx])

    response = llm.chat(
        model="gpt-4-1106-preview",
        content= prompt,
        system_instruct = "You are a helpful assistant."
        )
    
    if '[Review Begin]' in response:
        review = response.split('[Review Begin]')[-1]
    elif 'Part 2, Review:' in response:
        review = response.split('Part 2, Review:')[-1]
    elif 'Part 2, Review.' in response:
        review = response.split('Part 2, Review.')[-1]
    elif 'Part 2, Review\n' in response:
        review = response.split('Part 2, Review\n')[-1]
    else:
        print(f'{idx}-th sample need help')
        review = response


    reviews.append(review)
    time.sleep(1)

chatcmpl-8nIuOTBVwmfFBZSD0r5AMyVCqaLIY
chatcmpl-8nIuSiws9faZYRZvBMibuC1j6eA2E
chatcmpl-8nIuXLpBzq5MFr5W62I8pY6Rn5uA2
chatcmpl-8nIuecPtXejkWJPjJEmw1zrSlUSyr
chatcmpl-8nIuiWc9Lsrt9nT5qzeGPycfZSnGJ
chatcmpl-8nIuuKABkiVT0c3ulywA5VHQ5gwKZ
chatcmpl-8nIv1sovbUqVgH42167soLPHiG89J
chatcmpl-8nIv54eA1u0CxgiGBnvsGoVnJSIHy
chatcmpl-8nIv98DHeojtpkrgF3KU2UashdJPR
chatcmpl-8nIvHdoyMAlUATSJhhKu0YhhGyeQl
chatcmpl-8nIvMAZ1KjSkCTnZgEnswBeTWJ2MN
chatcmpl-8nIvUWoig92NY2Mt076Mp51sBmPsc
chatcmpl-8nIvfTnCynXCUYeDzmN2jbEo9gu6f
chatcmpl-8nIvlUkraNrF8MZe0owpUlVNjj8YV
chatcmpl-8nIvt4yLVVk71V8JCEjigLHBWGYa6
chatcmpl-8nIvyJRxzhRfXkYxksuD2CEf3A0KE
chatcmpl-8nIw4uxbHndnYSKh908frkOIiD8C8
chatcmpl-8nIwA9e3SAlSQfsfnl8HtJ2KMBGI0
chatcmpl-8nIwFjpM3kFjKJ8za4g4jKnQXlkHM
chatcmpl-8nIwOWtm1W5X1pgj9SYMmqM2B66Nc
chatcmpl-8nIwcHSQ1GNfgaydUJqoun892xAm1
chatcmpl-8nIwgGQEnUU4z9dmtX4t6hBhGntEX
chatcmpl-8nIwkcFlHJinDKnOljAyOBnD797HD
chatcmpl-8nIwp9qVElqozbgLPMyUtgCrFVUB2
chatcmpl-8nIwunbTiP3ZkFKkLTtaGptle8mOB
chatcmpl-8nIx2eKOpZqnvWIA

In [474]:
import pandas as pd

pd_data = pd.DataFrame(values, columns=names)
pd_data['Review'] = reviews
pd_data.head()

,score,color,size,smell,taste,juiciness,recmd,Review
0,-2,1,0,-1,-1,-1,-1,"Despite a visually appealing and vibrant hue,..."
1,-1,1,1,-1,-1,1,-1,"Despite its brilliant hue and grandeur, this ..."
2,2,1,1,1,1,1,1,"This apple's vibrant, consistent hue and larg..."
3,0,1,-1,1,-1,-1,1,"With a vibrant, uniform color, this pint-size..."
4,-1,1,-1,-1,1,1,0,"Despite its vibrant, uniform hue, the apple's..."


In [500]:
pd_data.isna().any()

score        False
color        False
size         False
smell        False
taste        False
juiciness    False
recmd        False
Review       False
dtype: bool

In [501]:
pd_data.to_excel("~/COAT/AppleGastronome/Apple_Gastronome_AG7_v20240201.xlsx", index=False) 

In [502]:


score_set = pd_data['score'].unique()
score_set = score_set[np.argsort(score_set)]

sample_size = pd_data.shape[0]

sorted_data = '# Data\n'
for v in score_set:
    sorted_data +=  f'\n## Score: {v}\n'
    this_indeces = np.arange(sample_size)[pd_data['score'].values==v]
    for this_idx in this_indeces:
        this_review = pd_data.values[this_idx,-1].replace("\n", " ")
        
        sorted_data +=  f'\n{this_review}\n'

print(sorted_data)


# Data

## Score: -4

 Unfortunately, this batch of apples fails to meet quality standards. Their small size, extensive surface blemishes, and dry texture fall short of expectations. The off-putting musty scent suggests potential rot, and the sour taste overpowers any sweetness. Suppliers should beware; these apples could result in financial disappointment if offered to discerning customers.

 This particular apple, while boasting a striking and consistent hue, falls short of expectations due to its unappealing small stature, an off-putting musty odor suggesting decay, and a flavor profile that tilts disappointingly toward sourness. The lack of juiciness further detracts from its desirability, making it a risky choice for suppliers and a potential loss-inducer in the market.

## Score: -3

 This apple, regrettably, fails to deliver on multiple fronts, its diminutive stature only overshadowed by an off-putting musty undertone that hints at decay. The fruit's unbalanced profile skewing t